# CCASS - Project 3 - Athlete and Celebrity Dashboard
### Members – Chris Brownlee, Chan Ho Ahn, Angela Huynh, Samantha Cassidy, Skip Hobba

### Pre-Work: 
### Step 1
#### Download Three Sets of data, rename files, and store in Data directory
#### https://www.kaggle.com/parulpandey/forbes-highest-paid-athletes-19902019 - rename to "atheletes.csv"
#### https://www.kaggle.com/slayomer/forbes-celebrity-100-since-2005 - rename to "celebrity.csv"
#### https://www.kaggle.com/paultimothymooney/latitude-and-longitude-for-every-country-and-state - rename to "country_state.csv"

### Step 2
#### Create Project3 database in postgres database
#### Run queries.sql file to create table structure

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests, gmaps, os, re, datetime
from sqlalchemy import create_engine

In [2]:
rds_connection_string = "postgres:postgres@localhost:5432/Project3"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [3]:
# Confirm tables
engine.table_names()

['celebrities', 'country', 'athletes']

## Import and clean country data

In [4]:
# Load country file to a dataframe
country_data = "data/country_state.csv"
country_df = pd.read_csv(country_data)

#Print the header from the dataset
country_df.head()

,country_code,latitude,longitude,country,usa_state_code,usa_state_latitude,usa_state_longitude,usa_state
0,AD,42.546245,1.601554,Andorra,AK,63.588753,-154.493062,Alaska
1,AE,23.424076,53.847818,United Arab Emirates,AL,32.318231,-86.902298,Alabama
2,AF,33.939110,67.709953,Afghanistan,AR,35.201050,-91.831833,Arkansas
3,AG,17.060816,-61.796428,Antigua and Barbuda,AZ,34.048928,-111.093731,Arizona
4,AI,18.220554,-63.068615,Anguilla,CA,36.778261,-119.417932,California


In [5]:
#Drop state columns - they are not needed in the data set, nor do they align correctly with the data
country_df = country_df.drop(['usa_state_code'], axis=1)
country_df = country_df.drop(['usa_state_latitude'], axis=1)
country_df = country_df.drop(['usa_state_longitude'], axis=1)
country_df = country_df.drop(['usa_state'], axis=1)

#Print the header from the dataset
country_df.head()

,country_code,latitude,longitude,country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


## Import and clean athelete data

In [6]:
athelete_data = "data/atheletes.csv"
athelete_df = pd.read_csv(athelete_data)

#Print the header from the dataset
athelete_df.head()

,S.NO,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
0,1,Mike Tyson,USA,1,NaN,boxing,1990,28.6
1,2,Buster Douglas,USA,2,NaN,boxing,1990,26.0
2,3,Sugar Ray Leonard,USA,3,NaN,boxing,1990,13.0
3,4,Ayrton Senna,Brazil,4,NaN,auto racing,1990,10.0
4,5,Alain Prost,France,5,NaN,auto racing,1990,9.0


In [7]:
#Drop columns that are not needed in the data set
athelete_df = athelete_df.drop(['S.NO'], axis=1)

athelete_df.head()

,Name,Nationality,Current Rank,Previous Year Rank,Sport,Year,earnings ($ million)
0,Mike Tyson,USA,1,NaN,boxing,1990,28.6
1,Buster Douglas,USA,2,NaN,boxing,1990,26.0
2,Sugar Ray Leonard,USA,3,NaN,boxing,1990,13.0
3,Ayrton Senna,Brazil,4,NaN,auto racing,1990,10.0
4,Alain Prost,France,5,NaN,auto racing,1990,9.0


In [8]:
#Rename Columns to align with database
athelete_df.rename(columns={
                            'Name':'name', 
                            'Nationality':'nationality', 
                            'Current Rank':'current_rank',
                            'Previous Year Rank':'previous_year_rank',
                            'Sport':'sport',
                            'Year':'year',
                            'earnings ($ million)':'earnings_in_mm'
                            }, inplace=True)

athelete_df.head()

,name,nationality,current_rank,previous_year_rank,sport,year,earnings_in_mm
0,Mike Tyson,USA,1,NaN,boxing,1990,28.6
1,Buster Douglas,USA,2,NaN,boxing,1990,26.0
2,Sugar Ray Leonard,USA,3,NaN,boxing,1990,13.0
3,Ayrton Senna,Brazil,4,NaN,auto racing,1990,10.0
4,Alain Prost,France,5,NaN,auto racing,1990,9.0


In [9]:
# Update previous_year_rank - set non-numeric and null values to 0
athelete_df['previous_year_rank'] = (
    pd.to_numeric(athelete_df['previous_year_rank'],
                  errors='coerce')
      .fillna(0)
    )

athelete_df['previous_year_rank'] = athelete_df['previous_year_rank'].astype(str).replace('\.0', '', regex=True)

athelete_df.head()

,name,nationality,current_rank,previous_year_rank,sport,year,earnings_in_mm
0,Mike Tyson,USA,1,0,boxing,1990,28.6
1,Buster Douglas,USA,2,0,boxing,1990,26.0
2,Sugar Ray Leonard,USA,3,0,boxing,1990,13.0
3,Ayrton Senna,Brazil,4,0,auto racing,1990,10.0
4,Alain Prost,France,5,0,auto racing,1990,9.0


In [10]:
# Display nationalities that are not perfectly linked to the country data
athelete_df[~athelete_df['nationality'].isin(country_df ['country'])].nationality.unique()

array(['USA', 'UK', 'Dominican', 'Filipino', 'Northern Ireland'],
      dtype=object)

In [11]:
# Update Nationality name to align with country name for later joins
athelete_df['nationality'] = athelete_df['nationality'].replace('USA', 'United States')
athelete_df['nationality'] = athelete_df['nationality'].replace('UK', 'United Kingdom')
athelete_df['nationality'] = athelete_df['nationality'].replace('Dominican', 'Dominican Republic')
athelete_df['nationality'] = athelete_df['nationality'].replace('Filipino', 'Philippines')
athelete_df['nationality'] = athelete_df['nationality'].replace('Northern Ireland', 'United Kingdom')

In [12]:
# Check to determine if nationalities that are all aligned
athelete_df[~athelete_df['nationality'].isin(country_df ['country'])].nationality.unique()

array([], dtype=object)

In [13]:
athelete_df.sport.unique()

array(['boxing', 'auto racing', 'golf', 'basketball', 'Basketball',
       'Boxing', 'Auto Racing', 'Golf', 'Tennis', 'NFL', 'Auto racing',
       'NBA', 'Baseball', 'Ice Hockey', 'American Football / Baseball',
       'tennis', 'ice hockey', 'F1 Motorsports', 'NASCAR', 'Hockey',
       'Auto Racing (Nascar)', 'F1 racing', 'American Football', 'soccer',
       'baseball', 'cycling', 'motorcycle gp', 'Soccer', 'MMA'],
      dtype=object)

In [14]:
#Update sport columns in the dataframe and standardize the category
athelete_df['sport'] = athelete_df['sport'].replace('American Football', 'Football')
athelete_df['sport'] = athelete_df['sport'].replace('American Football / Baseball', 'Baseball')
athelete_df['sport'] = athelete_df['sport'].replace('auto racing', 'Motorsports')
athelete_df['sport'] = athelete_df['sport'].replace('Auto racing', 'Motorsports')
athelete_df['sport'] = athelete_df['sport'].replace('Auto Racing', 'Motorsports')
athelete_df['sport'] = athelete_df['sport'].replace('Auto Racing (Nascar)', 'Motorsports')
athelete_df['sport'] = athelete_df['sport'].replace('baseball', 'Baseball')
athelete_df['sport'] = athelete_df['sport'].replace('basketball', 'Basketball')
athelete_df['sport'] = athelete_df['sport'].replace('boxing', 'Boxing')
athelete_df['sport'] = athelete_df['sport'].replace('cycling', 'Cycling')
athelete_df['sport'] = athelete_df['sport'].replace('F1 Motorsports', 'Motorsports')
athelete_df['sport'] = athelete_df['sport'].replace('F1 racing', 'Motorsports')
athelete_df['sport'] = athelete_df['sport'].replace('golf', 'Golf')
athelete_df['sport'] = athelete_df['sport'].replace('ice hockey', 'Hockey')
athelete_df['sport'] = athelete_df['sport'].replace('Ice Hockey', 'Hockey')
athelete_df['sport'] = athelete_df['sport'].replace('MMA', 'MMA')
athelete_df['sport'] = athelete_df['sport'].replace('motorcycle gp', 'Motorsports')
athelete_df['sport'] = athelete_df['sport'].replace('NASCAR', 'Motorsports')
athelete_df['sport'] = athelete_df['sport'].replace('NBA', 'Basketball')
athelete_df['sport'] = athelete_df['sport'].replace('NFL', 'Football')
athelete_df['sport'] = athelete_df['sport'].replace('soccer', 'Soccer')
athelete_df['sport'] = athelete_df['sport'].replace('tennis', 'Tennis')

In [15]:
athelete_df.sport.unique()

array(['Boxing', 'Motorsports', 'Golf', 'Basketball', 'Tennis',
       'Football', 'Baseball', 'Hockey', 'Soccer', 'Cycling', 'MMA'],
      dtype=object)

In [16]:
#Display new dataframe
athelete_df.head()

,name,nationality,current_rank,previous_year_rank,sport,year,earnings_in_mm
0,Mike Tyson,United States,1,0,Boxing,1990,28.6
1,Buster Douglas,United States,2,0,Boxing,1990,26.0
2,Sugar Ray Leonard,United States,3,0,Boxing,1990,13.0
3,Ayrton Senna,Brazil,4,0,Motorsports,1990,10.0
4,Alain Prost,France,5,0,Motorsports,1990,9.0


## Import and clean celebrity data

In [17]:
# Load Celebrity data files into a dataframe
celebrities_data = "data/celebrity.csv"
celebrities_df = pd.read_csv(celebrities_data)

#Print the header from the dataset
celebrities_df.head()

,Name,Pay (USD millions),Year,Category
0,Oprah Winfrey,225.0,2005,Personalities
1,Tiger Woods,87.0,2005,Athletes
2,Mel Gibson,185.0,2005,Actors
3,George Lucas,290.0,2005,Directors/Producers
4,Shaquille O'Neal,33.4,2005,Athletes


In [18]:
#Rename Columns to align with database table
celebrities_df.rename(columns={
                            'Name':'name', 
                            'Pay (USD millions)':'pay_in_mm', 
                            'Year':'year',
                            'Category':'category'
                            }, inplace=True)

In [19]:
celebrities_df.category.unique()

array(['Personalities', 'Athletes', 'Actors', 'Directors/Producers',
       'Musicians', 'Authors', 'Comedians', 'Television actresses',
       'Actresses', 'Magicians', 'Models', 'Television actors',
       'Hip-hop impresario'], dtype=object)

In [20]:
#Update category columns in the dataframe and standardize the category
celebrities_df['category'] = celebrities_df['category'].replace('Actors', 'Actors Actresses')
celebrities_df['category'] = celebrities_df['category'].replace('Authors', 'Authors')
celebrities_df['category'] = celebrities_df['category'].replace('Television actresses', 'Actors Actresses')
celebrities_df['category'] = celebrities_df['category'].replace('Actresses', 'Actors Actresses')
celebrities_df['category'] = celebrities_df['category'].replace('Television actors', 'Actors Actresses')
celebrities_df['category'] = celebrities_df['category'].replace('Hip-hop impresario', 'Musicians')

In [21]:
celebrities_df.category.unique()

array(['Personalities', 'Athletes', 'Actors Actresses',
       'Directors/Producers', 'Musicians', 'Authors', 'Comedians',
       'Magicians', 'Models'], dtype=object)

In [24]:
#remove athletes from the celbrity dataframe
celebrities_df = celebrities_df[celebrities_df.category != 'Athletes']

In [25]:
celebrities_df.category.unique()

array(['Personalities', 'Actors Actresses', 'Directors/Producers',
       'Musicians', 'Authors', 'Comedians', 'Magicians', 'Models'],
      dtype=object)

In [26]:
celebrities_df.head()

,name,pay_in_mm,year,category
0,Oprah Winfrey,225.0,2005,Personalities
2,Mel Gibson,185.0,2005,Actors Actresses
3,George Lucas,290.0,2005,Directors/Producers
5,Steven Spielberg,80.0,2005,Directors/Producers
6,Johnny Depp,37.0,2005,Actors Actresses


## Load clean data into created database tables

In [27]:
# Load Dataframes to SQL tables
athelete_df.to_sql(name='athletes', con=engine, if_exists='append', index=False)
celebrities_df.to_sql(name='celebrities', con=engine, if_exists='append', index=False)
country_df.to_sql(name='country', con=engine, if_exists='append', index=False)

In [31]:
athelete_df.to_csv('clean_athelete.csv', index=False)
celebrities_df.to_csv('data/clean_celebrities.csv', index=False)
country_df.to_csv('data/clean_country.csv', index=False)

In [28]:
#Check to see if data loaded correctly into the SQL tables
pd.read_sql_query('select * from athletes', con=engine).head()

,name,nationality,current_rank,previous_year_rank,sport,year,earnings_in_mm
0,Mike Tyson,United States,1,0,Boxing,1990,29
1,Buster Douglas,United States,2,0,Boxing,1990,26
2,Sugar Ray Leonard,United States,3,0,Boxing,1990,13
3,Ayrton Senna,Brazil,4,0,Motorsports,1990,10
4,Alain Prost,France,5,0,Motorsports,1990,9


In [29]:
pd.read_sql_query('select * from celebrities', con=engine).head()

,name,pay_in_mm,year,category
0,Oprah Winfrey,225,2005,Personalities
1,Mel Gibson,185,2005,Actors Actresses
2,George Lucas,290,2005,Directors/Producers
3,Steven Spielberg,80,2005,Directors/Producers
4,Johnny Depp,37,2005,Actors Actresses


In [30]:
pd.read_sql_query('select * from country', con=engine).head()

,country_code,latitude,longitude,country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
